In [2]:
from datetime import datetime as dt, timedelta
from dotenv import load_dotenv # QUANDO NOS TIVERMOS O COFRE DE SEMHAS TROCAREMOS ESSA LIB PELA LIB DE CONSUMO DE CHAVES DA AWS
from bs4 import BeautifulSoup
from app_s3 import S3 # LIB AWS

import pandas as pd
import datetime
import requests
import os

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 10000)

In [3]:
# NOME DO ARQUIVO
nm_arq = 'outcome'

# Credenciais de autenticação
username = "labutare-bi"
password = "toYqCZweFTzX8AikcPAu4IspSxrsZtdN"

# Lista de endpoints
url = "https://api.sienge.com.br/labutare/public/api/bulk-data/v1/outcome"

# Parâmetros para endpoints que exigem filtros
params = {
    "startDate": "2020-01-01",
    "endDate": format(dt.now(), '%Y-%m-%d'),
    "selectionType": "I",
    "correctionIndexerId": "1",
    "correctionDate": format(dt.now(), '%Y-%m-%d'),
    "billsIds": "20"
}

# Loop sobre os endpoints
response = requests.get(url, auth=(username, password), params=params if "bulk-data" in url else None)

In [7]:
if response.status_code == 200:
    data = response.json()['data']

    df = pd.json_normalize(data)

In [8]:
def explode_and_normalize(df, explode_columns):

    df_result = df.copy()

    for col in explode_columns:
        df_exploded = df_result.explode(col)
        df_final = pd.json_normalize(df_exploded[col])

        l_colunas = [f'{col}_{u}' for u in list(df_final.columns)]
        df_final.rename(columns=dict(zip(df_final.columns, l_colunas)), inplace=True)

        df_result = pd.concat([df_exploded.reset_index(drop=True), df_final], axis=1).drop(columns=[col])

    return df_result

explode_columns = ["paymentsCategories", "departamentsCosts", "buildingsCosts", "payments", "authorizations",
                   "payments_bankMovements","payments_bankMovements_paymentCategories"]
df_geral = explode_and_normalize(df, explode_columns)

In [9]:
df_geral.to_csv("teste_outcome.csv", sep=';', encoding='utf-8-sig', decimal=',', index=False)